In [1378]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import codecs
import os
import gzip
import sys
from bs4 import BeautifulSoup
from itertools import chain        
from contextlib import ExitStack 
import time
import json
from sklearn.metrics import silhouette_samples
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
from multiprocessing.dummy import Pool as ThreadPool 
from multiprocessing.dummy import Lock as ThreadLock 
from multiprocessing.dummy import Value as ThreadValue
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
import nltk
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import DBSCAN
import pymorphy2
from scipy.spatial.distance import cdist
from scipy.spatial.distance import cosine
from multiprocessing.dummy import Lock
from gensim import corpora
from gensim.summarization import bm25
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer
from pytorch_transformers import BertTokenizer, BertConfig
from transformers import  BertModel
from sklearn.metrics.pairwise import cosine_similarity

In [390]:
morph = pymorphy2.MorphAnalyzer()

In [391]:
stemmerR = SnowballStemmer('russian')
stemmerE = SnowballStemmer('english')

In [392]:
def inp_for_titles(line):
    data = line.split('\t', 1)
    doc_id = int(data[0])
    if len(data[1]) == 1:
        title = ''
    else:
        title = data[1]
    doc_to_title[doc_id] = title[:-1]

In [393]:
doc_to_title = {}
with codecs.open('docs_titles.tsv', 'r', 'utf8') as f:
    next(f)
    with ThreadPool(10) as pool:
        pool.map(inp_for_titles, f)
print(len(doc_to_title))

28026


In [395]:
stop_words = set(stopwords.words(["russian", "english"]))

In [396]:
lock = Lock()

In [398]:
def Parser_for_titles(ID):
    title = (doc_to_title[ID])
    title = title.lower()
    words = [stemmerR.stem(stemmerE.stem(word)) for word in re.sub('[^a-zа-я0-9]', ' ', title).split()]
    with lock:
        vec[ID] = ' '.join([word for word in words])

In [399]:
def Parser_for_titles_without_stemming(ID):
    title = (doc_to_title[ID])
    title = title.lower()
    words = [word for word in re.sub('[^a-zа-я0-9]', ' ', title).split()]
    with lock:
        vec[ID] = ' '.join([word for word in words])

In [400]:
def Parser_for_text(ID):
    with lock:
        text_id = df[df.id == ID].text.values
    text_id = text_id[0]
    stemmed_text = [stemmerR.stem(stemmerE.stem(word)) for word in text_id.split(' ')[:512]]
    with lock:
        vector_of_text[str(ID)] = ' '.join([word for word in stemmed_text])

In [120]:
df_iter = pd.read_csv('/Users/egor/Desktop/Техносфера/MachineLearning/project/data/project.csv', encoding='utf-8', chunksize=500)

In [159]:
i = 0
for df in df_iter:
    df.fillna(' ')
    vector_of_text = {}
    idx = df.id.values
    with ThreadPool(10) as pool:
        pool.map(Parser_for_text, idx)
    with open('texts/text_stemmed_part_{:05d}.json'.format(i), mode='wb') as f_json:
        f_json = codecs.getwriter('utf8')(f_json)
        record = json.dumps(vector_of_text, ensure_ascii=False)
        print(record, file=f_json)
    i += 1

есть 57 файлов с индексами страниц по 500

In [401]:
vec = {}
with ThreadPool(10) as pool:
    pool.map(Parser_for_titles_without_stemming, list(range(1,28027)))

In [403]:
with open('titles_without_stemming.json', 'w', encoding='utf8') as file:
    json.dump(vec, file)

In [27]:
with open('titles_stemming.json', 'w', encoding='utf8') as file:
    json.dump(vec, file)

Застемил заголовки - уже прекрасно!

## Мутим фичи

In [2]:
from gensim.models import FastText
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models.fasttext import load_facebook_model

Взяли заголовки

In [3]:
vec = {}

In [4]:
with open('titles_stemming.json', 'r', encoding='utf8') as file:
    vec = json.load(file)

In [5]:
sentences = []
for value in vec.values():
    sentences.append(value)

In [1577]:
model_fasttext = FastText('/Users/egor/Desktop/Техносфера/MachineLearning/project/ft_freqprune_400K_100K_pq_300.bin')

In [7]:
len(sentences)

28026

In [8]:
first = sentences[0]
first

'м б аншин центр репродукц и генетик фертимед г москв'

In [9]:
len(model_fasttext.wv[first])

100

Давайте попробуем сделать общий файл с эмеддингами текстов.. не просто так же их парсили

Делаем эмбединги и запоминаем их

In [40]:
filenames = os.listdir('texts_57_json')
texts_embeddings = {}
texts_embeding_to_json = {}
for f_name in filenames:
    file_n = 'texts_57_json/' + f_name
    with open(file_n, mode='r') as file:
        d = json.load(file)  # все тексты 
        for key, val in d.items():
            embedding = model_fasttext.wv[val]
            embedding_string = tuple(map(str, embedding))
            texts_embeddings[key] = embedding
            texts_embeding_to_json[key] = embedding_string

Хочу сделать Tf-Idf векторизацию

Запись в файл стринговых эмбеддингов

In [43]:
with open('texts_embeddings.json', 'w', encoding='utf8') as file:
    json.dump(texts_embeding_to_json, file)

Попробуем прочитать эмбеддинги - не забудь перевести в float

In [1367]:
with open('texts_embeddings.json', 'r', encoding='utf8') as file:
    texts_embedding_from_json = json.load(file)

In [1368]:
for key, val in texts_embedding_from_json.items():
    embedding = np.array(list(map(float, texts_embedding_from_json[key])))
    texts_embedding_from_json[key] = embedding

## Фичи

Если фасттекст применять к текстам после стемминга, результат получается лучше почему-то.

Для заголовков:
1) Метки кластеров внутри группы
2) Силуэт для объекта группы

In [1605]:
class Import:
    def __init__(self):
        self.doc_to_title = {}
        #self.model_fasttext = FastText('/Users/egor/Desktop/Техносфера/MachineLearning/project/ft_freqprune_400K_100K_pq_300.bin')
        with open('/Users/egor/Desktop/Техносфера/MachineLearning/project/docs_titles.tsv') as f:
            for num_line, line in enumerate(f):
                if num_line == 0:
                    continue
                data = line.strip().split('\t', 1)
                doc_id = int(data[0])
                if len(data) == 1:
                    title = ''
                else:
                    title = data[1]
                self.doc_to_title[doc_id] = title
        #self.top_k_cosine_titles = 3 # k ближайших косинусных расстояний для заголовков
        #self.best_docs_cnt = 10 # K самых похожих заголовков
        self.top_k_cosine_tf_idf = 10
        vec['0'] = ''
        self.doc_to_vec = TfidfVectorizer().fit_transform([vec[str(i)] for i in range(len(vec))])
        #self.top_k_cosine_texts = 20 # k ближайших косинусных расстояний для заголовков
    
    def make_train(self):
        train_data = pd.read_csv('/Users/egor/Desktop/Техносфера/MachineLearning/project/train_groups.csv')
        traingroups_titledata = {}
        for i in range(len(train_data)):
            new_doc = train_data.iloc[i]
            doc_group = new_doc['group_id']
            doc_id = new_doc['doc_id']
            target = new_doc['target']
            title = self.doc_to_title[doc_id]
            if doc_group not in traingroups_titledata:
                traingroups_titledata[doc_group] = []
            traingroups_titledata[doc_group].append((doc_id, title, target))
        
        y_train = []
        X_train = []
        groups_train = []
        for new_group in traingroups_titledata:
            docs = traingroups_titledata[new_group]
            for k, (doc_id, title, target_id) in enumerate(docs):
                y_train.append(target_id)
                groups_train.append(new_group)
                all_dist = []
                words = set(title.strip().split())
                for j in range(0, len(docs)):
                    if k == j:
                        continue
                    doc_id_j, title_j, target_j = docs[j]
                    words_j = set(title_j.strip().split())
                    all_dist.append(len(words.intersection(words_j)))
                X_train.append(sorted(all_dist, reverse=True)[0:15]    )
        X_train = np.array(X_train)
        y_train = np.array(y_train)
        groups_train = np.array(groups_train)
        
        # Класстеризация внутри группы:
        i = 0
        My_X = np.zeros((train_data.shape[0], self.top_k_cosine_tf_idf)) # кол-во новых фичей
        groups = train_data.groupby('group_id')
        rand_index = []
        for group_id, group_indx in groups.groups.items():
            group = train_data.iloc[group_indx]
            docs_id = group.doc_id.values
            # сначала заберем все файлы из группы
            embedded_texts = []
            group_titles = []
            for idx in docs_id:
                group_titles.append(vec[str(idx)])
                embedded_texts.append(texts_embedding_from_json[str(idx)])
                
            
            '''
            # BM25
            best_docs = []
            texts = [doc.split() for doc in group_titles]
            dictionary = corpora.Dictionary(texts)
            corpus = [dictionary.doc2bow(text) for text in texts]
            bm25_obj = bm25.BM25(corpus)
            for title in group_titles:
                query = title
                query_doc = dictionary.doc2bow(query.split())
                scores = bm25_obj.get_scores(query_doc)
                best_docs.append(sorted(range(len(scores)), key=lambda i: scores[i])[-self.best_docs_cnt:])
            '''
            #
            embedded_titles = model_fasttext.wv[group_titles]
            tf_idf_vec = self.doc_to_vec[group.doc_id]
            targets = group.target.values
            model_titles = DBSCAN(0.6, metric="cosine", min_samples=8)
            labels_titles = model_titles.fit_predict(tf_idf_vec)
            '''
            if np.unique(labels_titles).size > 1:
                silhouette = silhouette_samples(embedded_titles, labels_titles)
            else:
                silhouette = np.ones(len(labels_titles)) # если всего один кластер
            model_texts = DBSCAN(0.1, metric="cosine", min_samples=4)
            labels_texts = model_texts.fit_predict(embedded_texts)
            # добавим фичи из расстояний для эмбеддингов
            # получим матрицу из попарных расстояний
            cos_distances = cosine_similarity(embedded_titles)
            sorted_dist = np.apply_along_axis(np.sort, axis=1, arr=cos_distances)[:, ::-1]
            cos_features = sorted_dist[:, 1: self.top_k_cosine_titles + 1]
            '''
            # добавим фичи из расстояний для tf-idf
            cos_distances = cosine_similarity(tf_idf_vec)
            sorted_dist = np.apply_along_axis(np.sort, axis=1, arr=cos_distances)[:, ::-1]
            tf_idf_features = sorted_dist[:, 1: self.top_k_cosine_tf_idf + 1]
            
            
            My_X[i: i + group_indx.size, :self.top_k_cosine_tf_idf] = tf_idf_features
            #My_X[i: i + group_indx.size, -1] = labels_titles
            #My_X[i: i + group_indx.size, self.top_k_cosine_tf_idf: 
            #     self.top_k_cosine_tf_idf + self.top_k_cosine_titles] = cos_features
            # валидация дала DBSCAN(0.6, metric="cosine", min_samples=8)
            #My_X[i: i + group_indx.size, -1] = labels_texts
            i += group_indx.size
        
        
        X_train = np.concatenate((X_train, My_X), axis=1)
        return X_train, y_train, groups_train
    
    def make_test(self):
        test_data = pd.read_csv('/Users/egor/Desktop/Техносфера/MachineLearning/project/test_groups.csv')
        testgroups_titledata = {}
        for i in range(len(test_data)):
            new_doc = test_data.iloc[i]
            doc_group = new_doc['group_id']
            doc_id = new_doc['doc_id']
            title = self.doc_to_title[doc_id]
            if doc_group not in testgroups_titledata:
                testgroups_titledata[doc_group] = []
            testgroups_titledata[doc_group].append((doc_id, title))
            
        X_test = []
        groups_test = []
        for new_group in testgroups_titledata:
            docs = testgroups_titledata[new_group]
            for k, (doc_id, title) in enumerate(docs):
                groups_test.append(new_group)
                all_dist = []
                words = set(title.strip().split())
                for j in range(0, len(docs)):
                    if k == j:
                        continue
                    doc_id_j, title_j = docs[j]
                    words_j = set(title_j.strip().split())
                    all_dist.append(len(words.intersection(words_j)))
                X_test.append(sorted(all_dist, reverse=True)[0:15])
        X_test = np.array(X_test)
        
        i = 0
        My_X = np.zeros((test_data.shape[0], self.top_k_cosine_tf_idf)) # кол-во новых фичей
        groups = test_data.groupby('group_id')
        for group_id, group_indx in groups.groups.items():
            group = test_data.iloc[group_indx]
            docs_id = group.doc_id.values
            group_titles = []
            embedded_texts = []
            for idx in docs_id:
                group_titles.append(vec[str(idx)])
                embedded_texts.append(texts_embedding_from_json[str(idx)])
                
            
            embedded_titles = model_fasttext.wv[group_titles]
            tf_idf_vec = self.doc_to_vec[group.doc_id]
            model_titles = DBSCAN(0.6, metric="cosine", min_samples=8)
            labels_titles = model_titles.fit_predict(tf_idf_vec)
            '''
            if np.unique(labels_titles).size > 1:
                silhouette = silhouette_samples(embedded_titles, labels_titles)
            else:
                silhouette = np.ones(len(labels_titles)) # если всего один кластер
            model_texts = DBSCAN(0.1, metric="cosine", min_samples=4)
            labels_texts = model_texts.fit_predict(embedded_texts)
            cos_distances = cosine_similarity(embedded_titles)
            sorted_dist = np.apply_along_axis(np.sort, axis=1, arr=cos_distances)[:, ::-1]
            cos_features = sorted_dist[:, 1: self.top_k_cosine_titles + 1]
            '''
            # добавим фичи из расстояний для tf-idf
            cos_distances = cosine_similarity(tf_idf_vec)
            sorted_dist = np.apply_along_axis(np.sort, axis=1, arr=cos_distances)[:, ::-1]
            tf_idf_features = sorted_dist[:, 1: self.top_k_cosine_tf_idf + 1]
            
            
            My_X[i: i + group_indx.size, :self.top_k_cosine_tf_idf] = tf_idf_features
            My_X[i: i + group_indx.size, -1] = labels_titles
            #My_X[i: i + group_indx.size, self.top_k_cosine_tf_idf: 
            #     self.top_k_cosine_tf_idf + self.top_k_cosine_titles] = cos_features
            #My_X[i: i + group_indx.size, :self.top_k_cosine_titles] = cos_features
            #My_X[i: i + group_indx.size, -1] = labels_texts
            i += group_indx.size
            
            #My_X[i: i + group_indx.size, self.top_k_cosine_titles:self.top_k_cosine_titles
            #    + self.top_k_cosine_texts] = cos_features_texts
            # валидация дала DBSCAN(0.6, metric="cosine", min_samples=8)
        
        X_test = np.concatenate((X_test, My_X), axis=1)
        groups_test = np.array(groups_test)
        return X_test, groups_test, test_data['pair_id'].tolist()

### Валидация для кластеризации

Валидация для заголовков дала DBSCAN(0.6, metric="cosine", min_samples=8)

In [1606]:
importer = Import()

In [1607]:
X_train, y_train, groups_train = importer.make_train()

In [1608]:
X_test, groups_test, pair_ids = importer.make_test()

In [1609]:
X_train.shape

(11690, 25)

In [1610]:
X_test.shape

(16627, 25)

## Модельки

In [1648]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
from sklearn.base import BaseEstimator
from statistics import mean
from gensim.models import FastText
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neural_network import MLPClassifier
import json
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import catboost
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [1612]:
my_sk = StandardScaler()
X_train_sk = my_sk.fit_transform(X_train)
X_test_sk = my_sk.fit_transform(X_test)

In [1579]:
def feature_generator(X_train, X_test, num_model):
    indices = np.array([])
    length = X_train.shape[1]
    features = np.arange(length)
    np.random.shuffle(features)
    for i in range(num_model):
        ind = np.random.randint(0, length, length)
        ind = np.unique(np.append(ind, features[i * (length // num_model) : (i + 1) * (length // num_model + 1)]))
        yield X_train[:, ind], X_test[:, ind]

In [1580]:
clf_e_1 = RandomForestClassifier(max_depth=10, n_estimators=150)
clf_e_2 = ExtraTreesClassifier(max_depth=15)
clf_e_4 = catboost.CatBoostClassifier()

In [1581]:
clf_e_1.fit(X_train_sk, y_train)
clf_e_2.fit(X_train_sk, y_train)
clf_e_4.fit(X_train_sk, y_train)

Learning rate set to 0.029437
0:	learn: 0.6666943	total: 38ms	remaining: 38s
1:	learn: 0.6423993	total: 63.4ms	remaining: 31.7s
2:	learn: 0.6194522	total: 79.2ms	remaining: 26.3s
3:	learn: 0.6002347	total: 94.8ms	remaining: 23.6s
4:	learn: 0.5819813	total: 109ms	remaining: 21.6s
5:	learn: 0.5644210	total: 123ms	remaining: 20.3s
6:	learn: 0.5483972	total: 136ms	remaining: 19.3s
7:	learn: 0.5335571	total: 151ms	remaining: 18.8s
8:	learn: 0.5202169	total: 166ms	remaining: 18.3s
9:	learn: 0.5069619	total: 179ms	remaining: 17.7s
10:	learn: 0.4955533	total: 194ms	remaining: 17.5s
11:	learn: 0.4842366	total: 216ms	remaining: 17.8s
12:	learn: 0.4744789	total: 243ms	remaining: 18.5s
13:	learn: 0.4672783	total: 257ms	remaining: 18.1s
14:	learn: 0.4607238	total: 277ms	remaining: 18.2s
15:	learn: 0.4528940	total: 298ms	remaining: 18.3s
16:	learn: 0.4456133	total: 312ms	remaining: 18s
17:	learn: 0.4394871	total: 326ms	remaining: 17.8s
18:	learn: 0.4341537	total: 340ms	remaining: 17.6s
19:	learn: 0.

170:	learn: 0.3248202	total: 3.17s	remaining: 15.4s
171:	learn: 0.3246734	total: 3.19s	remaining: 15.4s
172:	learn: 0.3245111	total: 3.21s	remaining: 15.4s
173:	learn: 0.3244481	total: 3.23s	remaining: 15.3s
174:	learn: 0.3243696	total: 3.25s	remaining: 15.3s
175:	learn: 0.3243448	total: 3.27s	remaining: 15.3s
176:	learn: 0.3242458	total: 3.29s	remaining: 15.3s
177:	learn: 0.3241891	total: 3.31s	remaining: 15.3s
178:	learn: 0.3240811	total: 3.32s	remaining: 15.2s
179:	learn: 0.3240061	total: 3.34s	remaining: 15.2s
180:	learn: 0.3238529	total: 3.36s	remaining: 15.2s
181:	learn: 0.3237294	total: 3.37s	remaining: 15.2s
182:	learn: 0.3236161	total: 3.4s	remaining: 15.2s
183:	learn: 0.3235421	total: 3.42s	remaining: 15.2s
184:	learn: 0.3233428	total: 3.44s	remaining: 15.2s
185:	learn: 0.3232672	total: 3.46s	remaining: 15.2s
186:	learn: 0.3232016	total: 3.48s	remaining: 15.1s
187:	learn: 0.3230278	total: 3.5s	remaining: 15.1s
188:	learn: 0.3228836	total: 3.52s	remaining: 15.1s
189:	learn: 0.

341:	learn: 0.3075945	total: 6.09s	remaining: 11.7s
342:	learn: 0.3074468	total: 6.11s	remaining: 11.7s
343:	learn: 0.3073100	total: 6.12s	remaining: 11.7s
344:	learn: 0.3071823	total: 6.13s	remaining: 11.6s
345:	learn: 0.3070372	total: 6.15s	remaining: 11.6s
346:	learn: 0.3068837	total: 6.17s	remaining: 11.6s
347:	learn: 0.3068019	total: 6.19s	remaining: 11.6s
348:	learn: 0.3066525	total: 6.21s	remaining: 11.6s
349:	learn: 0.3065113	total: 6.23s	remaining: 11.6s
350:	learn: 0.3064480	total: 6.25s	remaining: 11.6s
351:	learn: 0.3063428	total: 6.27s	remaining: 11.6s
352:	learn: 0.3062393	total: 6.29s	remaining: 11.5s
353:	learn: 0.3061732	total: 6.31s	remaining: 11.5s
354:	learn: 0.3060573	total: 6.33s	remaining: 11.5s
355:	learn: 0.3058541	total: 6.34s	remaining: 11.5s
356:	learn: 0.3057366	total: 6.37s	remaining: 11.5s
357:	learn: 0.3056237	total: 6.4s	remaining: 11.5s
358:	learn: 0.3054990	total: 6.42s	remaining: 11.5s
359:	learn: 0.3053494	total: 6.44s	remaining: 11.4s
360:	learn: 0

510:	learn: 0.2880700	total: 8.97s	remaining: 8.58s
511:	learn: 0.2879554	total: 8.98s	remaining: 8.56s
512:	learn: 0.2878767	total: 9s	remaining: 8.54s
513:	learn: 0.2878207	total: 9.02s	remaining: 8.53s
514:	learn: 0.2876968	total: 9.03s	remaining: 8.51s
515:	learn: 0.2876295	total: 9.05s	remaining: 8.49s
516:	learn: 0.2876183	total: 9.07s	remaining: 8.47s
517:	learn: 0.2874967	total: 9.09s	remaining: 8.45s
518:	learn: 0.2874707	total: 9.1s	remaining: 8.43s
519:	learn: 0.2873720	total: 9.11s	remaining: 8.41s
520:	learn: 0.2872854	total: 9.13s	remaining: 8.39s
521:	learn: 0.2871693	total: 9.14s	remaining: 8.37s
522:	learn: 0.2870352	total: 9.15s	remaining: 8.35s
523:	learn: 0.2869146	total: 9.16s	remaining: 8.32s
524:	learn: 0.2867571	total: 9.18s	remaining: 8.31s
525:	learn: 0.2867080	total: 9.2s	remaining: 8.29s
526:	learn: 0.2866828	total: 9.21s	remaining: 8.27s
527:	learn: 0.2865734	total: 9.23s	remaining: 8.25s
528:	learn: 0.2865622	total: 9.24s	remaining: 8.23s
529:	learn: 0.286

669:	learn: 0.2724116	total: 11.5s	remaining: 5.67s
670:	learn: 0.2723545	total: 11.6s	remaining: 5.66s
671:	learn: 0.2722210	total: 11.6s	remaining: 5.65s
672:	learn: 0.2721341	total: 11.6s	remaining: 5.63s
673:	learn: 0.2720110	total: 11.6s	remaining: 5.62s
674:	learn: 0.2720082	total: 11.6s	remaining: 5.61s
675:	learn: 0.2719230	total: 11.7s	remaining: 5.59s
676:	learn: 0.2718156	total: 11.7s	remaining: 5.58s
677:	learn: 0.2717152	total: 11.7s	remaining: 5.56s
678:	learn: 0.2715857	total: 11.8s	remaining: 5.56s
679:	learn: 0.2715603	total: 11.8s	remaining: 5.55s
680:	learn: 0.2714774	total: 11.8s	remaining: 5.54s
681:	learn: 0.2713780	total: 11.8s	remaining: 5.53s
682:	learn: 0.2712437	total: 11.9s	remaining: 5.51s
683:	learn: 0.2712414	total: 11.9s	remaining: 5.5s
684:	learn: 0.2711359	total: 11.9s	remaining: 5.48s
685:	learn: 0.2710874	total: 11.9s	remaining: 5.47s
686:	learn: 0.2709700	total: 12s	remaining: 5.46s
687:	learn: 0.2709026	total: 12s	remaining: 5.44s
688:	learn: 0.270

828:	learn: 0.2575146	total: 16s	remaining: 3.3s
829:	learn: 0.2574336	total: 16s	remaining: 3.29s
830:	learn: 0.2573016	total: 16.1s	remaining: 3.27s
831:	learn: 0.2572324	total: 16.1s	remaining: 3.25s
832:	learn: 0.2571054	total: 16.1s	remaining: 3.23s
833:	learn: 0.2570077	total: 16.1s	remaining: 3.21s
834:	learn: 0.2569311	total: 16.1s	remaining: 3.19s
835:	learn: 0.2568297	total: 16.2s	remaining: 3.17s
836:	learn: 0.2567076	total: 16.2s	remaining: 3.15s
837:	learn: 0.2566145	total: 16.2s	remaining: 3.13s
838:	learn: 0.2564682	total: 16.3s	remaining: 3.12s
839:	learn: 0.2563831	total: 16.3s	remaining: 3.11s
840:	learn: 0.2562719	total: 16.3s	remaining: 3.09s
841:	learn: 0.2562144	total: 16.4s	remaining: 3.07s
842:	learn: 0.2560968	total: 16.4s	remaining: 3.05s
843:	learn: 0.2559897	total: 16.4s	remaining: 3.03s
844:	learn: 0.2559857	total: 16.4s	remaining: 3.02s
845:	learn: 0.2559098	total: 16.5s	remaining: 3s
846:	learn: 0.2558171	total: 16.5s	remaining: 2.98s
847:	learn: 0.255736

999:	learn: 0.2431712	total: 19.3s	remaining: 0us


In [1582]:
res = (clf_e_1.predict_proba(X_train_sk)[:, 1] + clf_e_2.predict_proba(X_train_sk)[: ,1]
+ clf_e_4.predict_proba(X_train_sk)[:, 1])/ 3

In [1583]:
for th in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
    print(f1_score(y_train, np.array(res > th, dtype=int)))

0.7218831734960768
0.8022626660108214
0.8348843392714703
0.8626720183486238
0.8769444016633298
0.8344549125168237
0.7709578127829079


In [1584]:
f1_score(y_train, np.array(res > 0.5, dtype=int))

0.8769444016633298

In [1585]:
votingC = VotingClassifier(estimators=[('rfc', clf_e_1), ('extc', clf_e_2), ('catboost', clf_e_4)], voting='soft', n_jobs=4)

votingC = votingC.fit(X_train_sk, y_train)

In [1586]:
vote = votingC.predict_proba(X_train_sk)[:, 1]
for th in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
    print(f1_score(y_train, np.array(vote > th, dtype=int)))

0.7215438290449192
0.7999017319739591
0.8332446808510638
0.8620146152743946
0.8755774561133354
0.8341455217610486
0.7680843329698291


In [1589]:
y_pls = np.array(votingC.predict_proba(X_test_sk)[:, 1] > 0.5, dtype=int)

In [1590]:
with open("/Users/egor/Desktop/subpobeda.csv", "w", encoding='utf-8') as outfile:
    outfile.write(str("pair_id,target\n"))
    for i in range(len(y_pls)):
        outfile.write(str(pair_ids[i]) + str(",") + str(y_pls[i]) + str("\n"))

Для DBSCAN(0.475, samples = 7):

0.82 при 5 соседях для косинуса

0.848095724500219 при 15 соседях

0.8509769612131818 при 20 соседях

0.8545427858505797 при 25 соседях

Для DBSCAN(0.6, samples = 7):

0.8305378831694621 на 5 соседях

0.8496447730897491 на 15 соседях

0.858557902403496 на 25 соседях

Min_samples = 8 дает получше скор

model_titles = DBSCAN(0.7, metric="cosine", min_samples=8):

На 25 соседях 0.8600523712540006

model_titles = DBSCAN(0.7, metric="cosine", min_samples=7) на 25 соседях дает 0.8593886462882097

model_titles = DBSCAN(0.7, metric="cosine", min_samples=9) на 25 соседях дает 0.8584289240598229

model_titles = DBSCAN(0.65, metric="cosine", min_samples=8) на 25 соседях дает 0.8568099053168244

model_titles = DBSCAN(0.6, metric="cosine", min_samples=8) на 25 соседях дает 0.8567264247194286

##### Добавил соседей по текстам:


DBSCAN(0.7, metric="cosine", min_samples=5)

Для 3 соседей 0.8733713951105255

Для 15 соседей 0.8819013876586951

Для 25 соседей 0.8833702882483371

##### model_titles = DBSCAN(0.65, metric="cosine", min_samples=8) на 25 соседях дает 0.8568099053168244 -  взял это, но количество соседей никак не учитывается

Для 3 соседей model_texts = DBSCAN(0.05, metric="cosine", min_samples=5) дает 0.857598607888631

Для 3 соседей model_texts = DBSCAN(0.15, metric="cosine", min_samples=5) дает 0.8583466511695481 

Для 3 соседей model_texts = DBSCAN(0.2, metric="cosine", min_samples=5) дает 0.8596746077861709

Для 3 соседей model_texts = DBSCAN(0.3, metric="cosine", min_samples=5) дает 0.8609368159929958

Для 3 соседей model_texts = DBSCAN(0.4, metric="cosine", min_samples=5) дает 0.8622248141128445 но это мне не нравится, потому что похоже на переобучение

Для 3 соседей model_texts = DBSCAN(0.4, metric="cosine", min_samples=6) дает 0.8609734771203731

Для 3 соседей model_texts = DBSCAN(0.4, metric="cosine", min_samples=4) дает 0.8611030055442077

##### Попрубую подключить одно косинусное расстояние для текстов, посмотрим, что будет

Для 1 соседей model_texts = DBSCAN(0.3, metric="cosine", min_samples=4) дает 0.8664050007268499

Для 2 соседей model_texts = DBSCAN(0.3, metric="cosine", min_samples=4) дает 0.8681672025723474

Для 20 соседей model_texts = DBSCAN(0.3, metric="cosine", min_samples=4) дает 0.880437999408109

Я не верю в расстояния между текстами!!!!

In [1513]:
test = (clf_e_1.predict_proba(X_test_sk)[:, 1] + clf_e_2.predict_proba(X_test_sk)[: ,1]
+ clf_e_4.predict_proba(X_test_sk)[:, 1])/ 3

In [1514]:
y_test = np.array(test > 0.5, dtype=int)

In [1515]:
with open("/Users/egor/Desktop/resultBest.csv", "w", encoding='utf-8') as outfile:
    outfile.write(str("pair_id,target\n"))
    for i in range(len(y_test)):
        outfile.write(str(pair_ids[i]) + str(",") + str(y_test[i]) + str("\n"))

### Лучше не использовать fasttext фичи

# Уже прям боевая валидация

In [1613]:
kfold = StratifiedKFold(n_splits=5)

In [1614]:
ExtC = ExtraTreesClassifier()


## Search grid for optimal parameters
ex_param_grid = {"max_depth": [None],
              "max_features": [1, 3, 10],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [False],
              "n_estimators" :[100,300],
              "criterion": ["gini"]}


gsExtC = GridSearchCV(ExtC,param_grid = ex_param_grid, cv=kfold, scoring="f1", n_jobs= 4, verbose = 1)

gsExtC.fit(X_train_sk,y_train)

ExtC_best = gsExtC.best_estimator_

# Best score
gsExtC.best_score_

Fitting 5 folds for each of 54 candidates, totalling 270 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   21.9s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:  1.7min
[Parallel(n_jobs=4)]: Done 270 out of 270 | elapsed:  3.2min finished


0.7233142062394696

In [1615]:
res_extra = ExtC_best.predict_proba(X_train_sk)[:, 1]
for th in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
    print(f1_score(y_train, np.array(res_extra > th, dtype=int)))

0.7316912972085386
0.8194977112458246
0.8700549818962049
0.9059112588524353
0.9059512796793093
0.8810184429574017
0.8109917209793905


In [1616]:
# RFC Parameters tunning 
RFC = RandomForestClassifier()


## Search grid for optimal parameters
rf_param_grid = {"max_depth": [None],
              "max_features": [1, 3, 10],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [False],
              "n_estimators" :[100,300],
              "criterion": ["gini"]}


gsRFC = GridSearchCV(RFC,param_grid = rf_param_grid, cv=kfold, scoring="f1", n_jobs= 4, verbose = 1)

gsRFC.fit(X_train_sk,y_train)

RFC_best = gsRFC.best_estimator_

# Best score
gsRFC.best_score_

Fitting 5 folds for each of 54 candidates, totalling 270 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   37.3s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:  4.3min
[Parallel(n_jobs=4)]: Done 270 out of 270 | elapsed: 10.3min finished


0.7210214368118383

In [1617]:
res_forest = RFC_best.predict_proba(X_train_sk)[:, 1]
for th in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
    print(f1_score(y_train, np.array(res_forest > th, dtype=int)))

0.740732543985836
0.8313403089187842
0.8862291554599248
0.9165945165945167
0.9215716251337716
0.8956283271495402
0.8236319275008713


In [1618]:
# Gradient boosting tunning

GBC = GradientBoostingClassifier()
gb_param_grid = {'loss' : ["deviance"],
              'n_estimators' : [100,200,300],
              'learning_rate': [0.1, 0.05, 0.01],
              'max_depth': [4, 8],
              'min_samples_leaf': [100,150],
              'max_features': [0.3, 0.1] 
              }

gsGBC = GridSearchCV(GBC,param_grid = gb_param_grid, cv=kfold, scoring="f1", n_jobs= 4, verbose = 1)

gsGBC.fit(X_train_sk,y_train)

GBC_best = gsGBC.best_estimator_

# Best score
gsGBC.best_score_

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   46.7s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:  3.4min
[Parallel(n_jobs=4)]: Done 360 out of 360 | elapsed:  6.2min finished


0.7226904892879523

In [1619]:
res_grad = GBC_best.predict_proba(X_train_sk)[:, 1]
for th in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
    print(f1_score(y_train, np.array(res_grad > th, dtype=int)))

0.6821523708044752
0.7382695307812313
0.7585216142092205
0.769209315297727
0.7599632127529123
0.7222880352005415
0.6601686972820994


In [1621]:
# Catboost tunning

CatBoost = catboost.CatBoostClassifier()
CB_param_grid = {'depth' : [3, 4, 5, 6],
              'l2_leaf_reg' : [21500, 22000, 25000, 28000, 28500],
              'scale_pos_weight': [1, 2, 2.4] 
              }

gsCatBoost = GridSearchCV(CatBoost,param_grid = CB_param_grid, cv=kfold, scoring="f1", n_jobs= 4, verbose = 1)

gsCatBoost.fit(X_train_sk, y_train)

CB_best = gsCatBoost.best_estimator_

# Best score
gsCatBoost.best_score_

Fitting 5 folds for each of 60 candidates, totalling 300 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  2.5min
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed: 11.7min
[Parallel(n_jobs=4)]: Done 300 out of 300 | elapsed: 20.6min finished


0:	learn: 0.6886435	total: 26.6ms	remaining: 26.6s
1:	learn: 0.6836696	total: 54.7ms	remaining: 27.3s
2:	learn: 0.6782128	total: 85.5ms	remaining: 28.4s
3:	learn: 0.6730009	total: 116ms	remaining: 28.8s
4:	learn: 0.6679825	total: 130ms	remaining: 25.9s
5:	learn: 0.6628277	total: 150ms	remaining: 24.9s
6:	learn: 0.6576212	total: 163ms	remaining: 23.1s
7:	learn: 0.6531237	total: 176ms	remaining: 21.9s
8:	learn: 0.6479266	total: 189ms	remaining: 20.8s
9:	learn: 0.6434655	total: 203ms	remaining: 20.1s
10:	learn: 0.6388841	total: 214ms	remaining: 19.2s
11:	learn: 0.6352815	total: 224ms	remaining: 18.5s
12:	learn: 0.6305905	total: 240ms	remaining: 18.2s
13:	learn: 0.6267094	total: 255ms	remaining: 18s
14:	learn: 0.6228047	total: 266ms	remaining: 17.5s
15:	learn: 0.6196926	total: 276ms	remaining: 17s
16:	learn: 0.6167993	total: 288ms	remaining: 16.7s
17:	learn: 0.6130989	total: 301ms	remaining: 16.4s
18:	learn: 0.6090044	total: 312ms	remaining: 16.1s
19:	learn: 0.6047858	total: 321ms	remainin

168:	learn: 0.4257992	total: 2.29s	remaining: 11.3s
169:	learn: 0.4255118	total: 2.32s	remaining: 11.3s
170:	learn: 0.4251893	total: 2.34s	remaining: 11.4s
171:	learn: 0.4248383	total: 2.36s	remaining: 11.4s
172:	learn: 0.4245239	total: 2.37s	remaining: 11.4s
173:	learn: 0.4242957	total: 2.39s	remaining: 11.3s
174:	learn: 0.4239862	total: 2.4s	remaining: 11.3s
175:	learn: 0.4236251	total: 2.42s	remaining: 11.3s
176:	learn: 0.4233266	total: 2.42s	remaining: 11.3s
177:	learn: 0.4230490	total: 2.44s	remaining: 11.2s
178:	learn: 0.4228286	total: 2.44s	remaining: 11.2s
179:	learn: 0.4225277	total: 2.46s	remaining: 11.2s
180:	learn: 0.4222696	total: 2.47s	remaining: 11.2s
181:	learn: 0.4219450	total: 2.48s	remaining: 11.1s
182:	learn: 0.4216164	total: 2.49s	remaining: 11.1s
183:	learn: 0.4213653	total: 2.51s	remaining: 11.1s
184:	learn: 0.4210706	total: 2.52s	remaining: 11.1s
185:	learn: 0.4207863	total: 2.53s	remaining: 11.1s
186:	learn: 0.4204496	total: 2.54s	remaining: 11.1s
187:	learn: 0

333:	learn: 0.4002897	total: 5.02s	remaining: 10s
334:	learn: 0.4002215	total: 5.04s	remaining: 10s
335:	learn: 0.4001596	total: 5.05s	remaining: 9.97s
336:	learn: 0.4000859	total: 5.06s	remaining: 9.95s
337:	learn: 0.4000158	total: 5.07s	remaining: 9.93s
338:	learn: 0.3999479	total: 5.08s	remaining: 9.9s
339:	learn: 0.3998726	total: 5.09s	remaining: 9.88s
340:	learn: 0.3997964	total: 5.1s	remaining: 9.86s
341:	learn: 0.3997398	total: 5.11s	remaining: 9.83s
342:	learn: 0.3996803	total: 5.12s	remaining: 9.81s
343:	learn: 0.3995922	total: 5.13s	remaining: 9.79s
344:	learn: 0.3995311	total: 5.14s	remaining: 9.76s
345:	learn: 0.3994638	total: 5.15s	remaining: 9.74s
346:	learn: 0.3993999	total: 5.16s	remaining: 9.72s
347:	learn: 0.3993411	total: 5.17s	remaining: 9.7s
348:	learn: 0.3992762	total: 5.18s	remaining: 9.67s
349:	learn: 0.3992182	total: 5.2s	remaining: 9.65s
350:	learn: 0.3991543	total: 5.21s	remaining: 9.63s
351:	learn: 0.3991009	total: 5.22s	remaining: 9.61s
352:	learn: 0.399022

497:	learn: 0.3918520	total: 7.05s	remaining: 7.11s
498:	learn: 0.3918236	total: 7.07s	remaining: 7.09s
499:	learn: 0.3917864	total: 7.08s	remaining: 7.08s
500:	learn: 0.3917555	total: 7.09s	remaining: 7.06s
501:	learn: 0.3917261	total: 7.1s	remaining: 7.04s
502:	learn: 0.3916895	total: 7.11s	remaining: 7.02s
503:	learn: 0.3916494	total: 7.12s	remaining: 7s
504:	learn: 0.3916131	total: 7.13s	remaining: 6.99s
505:	learn: 0.3915852	total: 7.14s	remaining: 6.97s
506:	learn: 0.3915577	total: 7.15s	remaining: 6.95s
507:	learn: 0.3915302	total: 7.16s	remaining: 6.93s
508:	learn: 0.3914929	total: 7.17s	remaining: 6.92s
509:	learn: 0.3914667	total: 7.19s	remaining: 6.91s
510:	learn: 0.3914314	total: 7.2s	remaining: 6.89s
511:	learn: 0.3913956	total: 7.22s	remaining: 6.88s
512:	learn: 0.3913568	total: 7.26s	remaining: 6.9s
513:	learn: 0.3913189	total: 7.28s	remaining: 6.88s
514:	learn: 0.3912933	total: 7.3s	remaining: 6.87s
515:	learn: 0.3912558	total: 7.32s	remaining: 6.86s
516:	learn: 0.39121

662:	learn: 0.3876215	total: 9.09s	remaining: 4.62s
663:	learn: 0.3876047	total: 9.11s	remaining: 4.61s
664:	learn: 0.3875890	total: 9.12s	remaining: 4.59s
665:	learn: 0.3875712	total: 9.13s	remaining: 4.58s
666:	learn: 0.3875480	total: 9.14s	remaining: 4.56s
667:	learn: 0.3875338	total: 9.15s	remaining: 4.55s
668:	learn: 0.3875099	total: 9.16s	remaining: 4.53s
669:	learn: 0.3874956	total: 9.17s	remaining: 4.52s
670:	learn: 0.3874805	total: 9.18s	remaining: 4.5s
671:	learn: 0.3874626	total: 9.19s	remaining: 4.49s
672:	learn: 0.3874473	total: 9.2s	remaining: 4.47s
673:	learn: 0.3874333	total: 9.21s	remaining: 4.45s
674:	learn: 0.3874180	total: 9.22s	remaining: 4.44s
675:	learn: 0.3874011	total: 9.23s	remaining: 4.42s
676:	learn: 0.3873859	total: 9.24s	remaining: 4.41s
677:	learn: 0.3873719	total: 9.25s	remaining: 4.39s
678:	learn: 0.3873495	total: 9.26s	remaining: 4.38s
679:	learn: 0.3873345	total: 9.27s	remaining: 4.36s
680:	learn: 0.3873195	total: 9.28s	remaining: 4.35s
681:	learn: 0.

839:	learn: 0.3851436	total: 11s	remaining: 2.09s
840:	learn: 0.3851337	total: 11s	remaining: 2.07s
841:	learn: 0.3851246	total: 11s	remaining: 2.06s
842:	learn: 0.3851148	total: 11s	remaining: 2.05s
843:	learn: 0.3851050	total: 11s	remaining: 2.04s
844:	learn: 0.3850936	total: 11s	remaining: 2.02s
845:	learn: 0.3850818	total: 11s	remaining: 2.01s
846:	learn: 0.3850708	total: 11s	remaining: 2s
847:	learn: 0.3850557	total: 11.1s	remaining: 1.98s
848:	learn: 0.3850461	total: 11.1s	remaining: 1.97s
849:	learn: 0.3850328	total: 11.1s	remaining: 1.95s
850:	learn: 0.3850179	total: 11.1s	remaining: 1.94s
851:	learn: 0.3850077	total: 11.1s	remaining: 1.93s
852:	learn: 0.3849935	total: 11.1s	remaining: 1.92s
853:	learn: 0.3849787	total: 11.1s	remaining: 1.9s
854:	learn: 0.3849700	total: 11.1s	remaining: 1.89s
855:	learn: 0.3849573	total: 11.2s	remaining: 1.88s
856:	learn: 0.3849486	total: 11.2s	remaining: 1.86s
857:	learn: 0.3849355	total: 11.2s	remaining: 1.85s
858:	learn: 0.3849263	total: 11.

NameError: name 'gsLogR' is not defined

In [1623]:
CB_best = gsCatBoost.best_estimator_

In [1624]:
res_catB = CB_best.predict_proba(X_train_sk)[:, 1]
for th in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
    print(f1_score(y_train, np.array(res_catB > th, dtype=int)))

0.617937049677664
0.6789383561643836
0.7002039428959891
0.7330012453300125
0.7423881387344452
0.7415062287655719
0.7136648961854177


In [1625]:
from sklearn.ensemble import StackingClassifier

In [1626]:
estimators = [('rf', RFC_best), ('et', ExtC_best), ('gb', GBC_best), ('cb', CB_best)]

In [1649]:
stack = StackingClassifier(estimators=estimators, final_estimator=MLPClassifier())

In [1650]:
stack.fit(X_train_sk, y_train)

0:	learn: 0.6886435	total: 10.8ms	remaining: 10.8s
1:	learn: 0.6836696	total: 22.5ms	remaining: 11.2s
2:	learn: 0.6782128	total: 31.8ms	remaining: 10.6s
3:	learn: 0.6730009	total: 43.3ms	remaining: 10.8s
4:	learn: 0.6679825	total: 53.2ms	remaining: 10.6s
5:	learn: 0.6628277	total: 64.4ms	remaining: 10.7s
6:	learn: 0.6576212	total: 74.8ms	remaining: 10.6s
7:	learn: 0.6531237	total: 86.1ms	remaining: 10.7s
8:	learn: 0.6479266	total: 96.5ms	remaining: 10.6s
9:	learn: 0.6434655	total: 107ms	remaining: 10.6s
10:	learn: 0.6388841	total: 117ms	remaining: 10.5s
11:	learn: 0.6352815	total: 128ms	remaining: 10.5s
12:	learn: 0.6305905	total: 139ms	remaining: 10.5s
13:	learn: 0.6267094	total: 148ms	remaining: 10.4s
14:	learn: 0.6228047	total: 160ms	remaining: 10.5s
15:	learn: 0.6196926	total: 170ms	remaining: 10.4s
16:	learn: 0.6167993	total: 180ms	remaining: 10.4s
17:	learn: 0.6130989	total: 190ms	remaining: 10.4s
18:	learn: 0.6090044	total: 201ms	remaining: 10.4s
19:	learn: 0.6047858	total: 215m

163:	learn: 0.4275097	total: 2.26s	remaining: 11.5s
164:	learn: 0.4270470	total: 2.27s	remaining: 11.5s
165:	learn: 0.4267540	total: 2.29s	remaining: 11.5s
166:	learn: 0.4263883	total: 2.3s	remaining: 11.5s
167:	learn: 0.4260844	total: 2.31s	remaining: 11.4s
168:	learn: 0.4257992	total: 2.32s	remaining: 11.4s
169:	learn: 0.4255118	total: 2.33s	remaining: 11.4s
170:	learn: 0.4251893	total: 2.37s	remaining: 11.5s
171:	learn: 0.4248383	total: 2.39s	remaining: 11.5s
172:	learn: 0.4245239	total: 2.4s	remaining: 11.5s
173:	learn: 0.4242957	total: 2.41s	remaining: 11.5s
174:	learn: 0.4239862	total: 2.42s	remaining: 11.4s
175:	learn: 0.4236251	total: 2.43s	remaining: 11.4s
176:	learn: 0.4233266	total: 2.45s	remaining: 11.4s
177:	learn: 0.4230490	total: 2.47s	remaining: 11.4s
178:	learn: 0.4228286	total: 2.5s	remaining: 11.5s
179:	learn: 0.4225277	total: 2.52s	remaining: 11.5s
180:	learn: 0.4222696	total: 2.53s	remaining: 11.5s
181:	learn: 0.4219450	total: 2.56s	remaining: 11.5s
182:	learn: 0.4

334:	learn: 0.4002215	total: 4.54s	remaining: 9.01s
335:	learn: 0.4001596	total: 4.55s	remaining: 9s
336:	learn: 0.4000859	total: 4.56s	remaining: 8.97s
337:	learn: 0.4000158	total: 4.58s	remaining: 8.97s
338:	learn: 0.3999479	total: 4.59s	remaining: 8.95s
339:	learn: 0.3998726	total: 4.6s	remaining: 8.92s
340:	learn: 0.3997964	total: 4.61s	remaining: 8.91s
341:	learn: 0.3997398	total: 4.63s	remaining: 8.91s
342:	learn: 0.3996803	total: 4.64s	remaining: 8.89s
343:	learn: 0.3995922	total: 4.66s	remaining: 8.88s
344:	learn: 0.3995311	total: 4.67s	remaining: 8.86s
345:	learn: 0.3994638	total: 4.68s	remaining: 8.85s
346:	learn: 0.3993999	total: 4.7s	remaining: 8.84s
347:	learn: 0.3993411	total: 4.71s	remaining: 8.83s
348:	learn: 0.3992762	total: 4.73s	remaining: 8.82s
349:	learn: 0.3992182	total: 4.76s	remaining: 8.84s
350:	learn: 0.3991543	total: 4.78s	remaining: 8.83s
351:	learn: 0.3991009	total: 4.79s	remaining: 8.82s
352:	learn: 0.3990223	total: 4.81s	remaining: 8.81s
353:	learn: 0.398

497:	learn: 0.3918520	total: 6.62s	remaining: 6.67s
498:	learn: 0.3918236	total: 6.64s	remaining: 6.66s
499:	learn: 0.3917864	total: 6.65s	remaining: 6.65s
500:	learn: 0.3917555	total: 6.66s	remaining: 6.64s
501:	learn: 0.3917261	total: 6.67s	remaining: 6.62s
502:	learn: 0.3916895	total: 6.69s	remaining: 6.61s
503:	learn: 0.3916494	total: 6.71s	remaining: 6.6s
504:	learn: 0.3916131	total: 6.72s	remaining: 6.59s
505:	learn: 0.3915852	total: 6.73s	remaining: 6.57s
506:	learn: 0.3915577	total: 6.74s	remaining: 6.55s
507:	learn: 0.3915302	total: 6.75s	remaining: 6.54s
508:	learn: 0.3914929	total: 6.76s	remaining: 6.52s
509:	learn: 0.3914667	total: 6.77s	remaining: 6.5s
510:	learn: 0.3914314	total: 6.78s	remaining: 6.49s
511:	learn: 0.3913956	total: 6.79s	remaining: 6.47s
512:	learn: 0.3913568	total: 6.8s	remaining: 6.45s
513:	learn: 0.3913189	total: 6.81s	remaining: 6.44s
514:	learn: 0.3912933	total: 6.82s	remaining: 6.42s
515:	learn: 0.3912558	total: 6.83s	remaining: 6.41s
516:	learn: 0.3

661:	learn: 0.3876374	total: 8.69s	remaining: 4.44s
662:	learn: 0.3876215	total: 8.71s	remaining: 4.43s
663:	learn: 0.3876047	total: 8.72s	remaining: 4.41s
664:	learn: 0.3875890	total: 8.73s	remaining: 4.4s
665:	learn: 0.3875712	total: 8.74s	remaining: 4.38s
666:	learn: 0.3875480	total: 8.75s	remaining: 4.37s
667:	learn: 0.3875338	total: 8.76s	remaining: 4.35s
668:	learn: 0.3875099	total: 8.78s	remaining: 4.34s
669:	learn: 0.3874956	total: 8.79s	remaining: 4.33s
670:	learn: 0.3874805	total: 8.8s	remaining: 4.32s
671:	learn: 0.3874626	total: 8.81s	remaining: 4.3s
672:	learn: 0.3874473	total: 8.82s	remaining: 4.29s
673:	learn: 0.3874333	total: 8.84s	remaining: 4.27s
674:	learn: 0.3874180	total: 8.84s	remaining: 4.26s
675:	learn: 0.3874011	total: 8.85s	remaining: 4.24s
676:	learn: 0.3873859	total: 8.87s	remaining: 4.23s
677:	learn: 0.3873719	total: 8.87s	remaining: 4.21s
678:	learn: 0.3873495	total: 8.88s	remaining: 4.2s
679:	learn: 0.3873345	total: 8.9s	remaining: 4.19s
680:	learn: 0.387

...

417:	learn: 0.4128469	total: 2.64s	remaining: 3.67s
418:	learn: 0.4127787	total: 2.65s	remaining: 3.67s
419:	learn: 0.4127138	total: 2.65s	remaining: 3.66s
420:	learn: 0.4126634	total: 2.66s	remaining: 3.65s
421:	learn: 0.4125973	total: 2.66s	remaining: 3.65s
422:	learn: 0.4125329	total: 2.67s	remaining: 3.64s
423:	learn: 0.4124779	total: 2.67s	remaining: 3.63s
424:	learn: 0.4124199	total: 2.68s	remaining: 3.63s
425:	learn: 0.4123601	total: 2.69s	remaining: 3.62s
426:	learn: 0.4122985	total: 2.69s	remaining: 3.61s
427:	learn: 0.4122415	total: 2.7s	remaining: 3.6s
428:	learn: 0.4121814	total: 2.71s	remaining: 3.6s
429:	learn: 0.4121210	total: 2.72s	remaining: 3.6s
430:	learn: 0.4120644	total: 2.73s	remaining: 3.6s
431:	learn: 0.4120075	total: 2.74s	remaining: 3.6s
432:	learn: 0.4119452	total: 2.75s	remaining: 3.6s
433:	learn: 0.4118875	total: 2.76s	remaining: 3.6s
434:	learn: 0.4118279	total: 2.77s	remaining: 3.59s
435:	learn: 0.4117721	total: 2.78s	remaining: 3.59s
436:	learn: 0.411725

597:	learn: 0.4052762	total: 3.83s	remaining: 2.57s
598:	learn: 0.4052470	total: 3.83s	remaining: 2.57s
599:	learn: 0.4052176	total: 3.84s	remaining: 2.56s
600:	learn: 0.4051840	total: 3.86s	remaining: 2.56s
601:	learn: 0.4051558	total: 3.86s	remaining: 2.55s
602:	learn: 0.4051267	total: 3.87s	remaining: 2.55s
603:	learn: 0.4050866	total: 3.88s	remaining: 2.54s
604:	learn: 0.4050580	total: 3.89s	remaining: 2.54s
605:	learn: 0.4050297	total: 3.9s	remaining: 2.54s
606:	learn: 0.4049966	total: 3.91s	remaining: 2.53s
607:	learn: 0.4049684	total: 3.92s	remaining: 2.53s
608:	learn: 0.4049395	total: 3.92s	remaining: 2.52s
609:	learn: 0.4049109	total: 3.93s	remaining: 2.51s
610:	learn: 0.4048831	total: 3.93s	remaining: 2.5s
611:	learn: 0.4048541	total: 3.94s	remaining: 2.5s
612:	learn: 0.4048269	total: 3.94s	remaining: 2.49s
613:	learn: 0.4047991	total: 3.95s	remaining: 2.48s
614:	learn: 0.4047715	total: 3.96s	remaining: 2.48s
615:	learn: 0.4047434	total: 3.96s	remaining: 2.47s
616:	learn: 0.4

785:	learn: 0.4008530	total: 5s	remaining: 1.36s
786:	learn: 0.4008346	total: 5.01s	remaining: 1.35s
787:	learn: 0.4008169	total: 5.02s	remaining: 1.35s
788:	learn: 0.4007999	total: 5.02s	remaining: 1.34s
789:	learn: 0.4007814	total: 5.03s	remaining: 1.34s
790:	learn: 0.4007571	total: 5.03s	remaining: 1.33s
791:	learn: 0.4007400	total: 5.04s	remaining: 1.32s
792:	learn: 0.4007231	total: 5.05s	remaining: 1.32s
793:	learn: 0.4007042	total: 5.06s	remaining: 1.31s
794:	learn: 0.4006802	total: 5.07s	remaining: 1.31s
795:	learn: 0.4006622	total: 5.07s	remaining: 1.3s
796:	learn: 0.4006452	total: 5.08s	remaining: 1.29s
797:	learn: 0.4006280	total: 5.08s	remaining: 1.29s
798:	learn: 0.4006114	total: 5.09s	remaining: 1.28s
799:	learn: 0.4005949	total: 5.1s	remaining: 1.27s
800:	learn: 0.4005762	total: 5.11s	remaining: 1.27s
801:	learn: 0.4005577	total: 5.12s	remaining: 1.26s
802:	learn: 0.4005392	total: 5.14s	remaining: 1.26s
803:	learn: 0.4005228	total: 5.16s	remaining: 1.26s
804:	learn: 0.400

962:	learn: 0.3981815	total: 6.17s	remaining: 237ms
963:	learn: 0.3981649	total: 6.18s	remaining: 231ms
964:	learn: 0.3981550	total: 6.19s	remaining: 225ms
965:	learn: 0.3981454	total: 6.2s	remaining: 218ms
966:	learn: 0.3981289	total: 6.21s	remaining: 212ms
967:	learn: 0.3981175	total: 6.21s	remaining: 205ms
968:	learn: 0.3981061	total: 6.22s	remaining: 199ms
969:	learn: 0.3980946	total: 6.23s	remaining: 193ms
970:	learn: 0.3980852	total: 6.24s	remaining: 186ms
971:	learn: 0.3980770	total: 6.24s	remaining: 180ms
972:	learn: 0.3980680	total: 6.25s	remaining: 173ms
973:	learn: 0.3980546	total: 6.25s	remaining: 167ms
974:	learn: 0.3980449	total: 6.26s	remaining: 161ms
975:	learn: 0.3980330	total: 6.27s	remaining: 154ms
976:	learn: 0.3980167	total: 6.27s	remaining: 148ms
977:	learn: 0.3980072	total: 6.28s	remaining: 141ms
978:	learn: 0.3979959	total: 6.28s	remaining: 135ms
979:	learn: 0.3979842	total: 6.29s	remaining: 128ms
980:	learn: 0.3979730	total: 6.29s	remaining: 122ms
981:	learn: 0

StackingClassifier(cv=None,
                   estimators=[('rf',
                                RandomForestClassifier(bootstrap=False,
                                                       ccp_alpha=0.0,
                                                       class_weight=None,
                                                       criterion='gini',
                                                       max_depth=None,
                                                       max_features=1,
                                                       max_leaf_nodes=None,
                                                       max_samples=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=3,
                                                       min_samples_split=2,
                                         

In [1651]:
stack_res = stack.predict_proba(X_train_sk)[:, 1]
for th in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
    print(f1_score(y_train, np.array(stack_res > th, dtype=int)))

0.6913659793814433
0.7452807060554058
0.759767687434002
0.7695955369595537
0.7710879734419798
0.735483870967742
0.6928738108059593


Нужно подобрать порог!!!

In [1640]:
y_pls = np.array(stack.predict_proba(X_test_sk)[:, 1] > 0.4, dtype=int)

In [1641]:
with open("/Users/egor/Desktop/subpobeda.csv", "w", encoding='utf-8') as outfile:
    outfile.write(str("pair_id,target\n"))
    for i in range(len(y_pls)):
        outfile.write(str(pair_ids[i]) + str(",") + str(y_pls[i]) + str("\n"))

на Кэгле = 0.68162 - сильное переобучение - решил убрать расстояние между текстами. Было 25 cosine, 0.65, 8; 0.3, 4

на Кэгле = 0.69587 без текстов но 25 cosine, 0.65, 8; 0.3, 4

На валидации 0.7809; на кэгле - 0.729. Это на 15 фичах от Севы и 10 tf-idf фичей

In [1632]:
from sklearn.ensemble import VotingClassifier

In [1652]:
votingC = VotingClassifier(estimators=[('rf', RFC_best), ('et', ExtC_best), ('gb', GBC_best), ('cb', CB_best),
                                       ('stack', stack)], voting='soft', n_jobs=4)

votingC = votingC.fit(X_train_sk, y_train)

In [1653]:
vote = votingC.predict_proba(X_train_sk)[:, 1]
for th in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
    print(f1_score(y_train, np.array(vote > th, dtype=int)))

0.6924516531503433
0.7530892982047096
0.7952544967470341
0.8075388636676296
0.8316212594908442
0.81343886286545
0.751706791232483


Валидация RandomForest + ExtraTree = 0.8157055925818604 на 25 фичах на titles и 3 на текстах. model_titles = DBSCAN(0.6, metric="cosine", min_samples=8); model_texts = DBSCAN(0.1, metric="cosine", min_samples=5)

In [1655]:
y_pls = np.array(votingC.predict_proba(X_test_sk)[:, 1] > 0.5, dtype=int)

In [1656]:
with open("/Users/egor/Desktop/subpobeda.csv", "w", encoding='utf-8') as outfile:
    outfile.write(str("pair_id,target\n"))
    for i in range(len(y_pls)):
        outfile.write(str(pair_ids[i]) + str(",") + str(y_pls[i]) + str("\n"))

на Кэгле 0.69022 текстов но 25 cosine, 0.65, 8; 0.1, 4

У Вани на фичах без текстов на рандомфоресте и экстраТри 0.71 на кагле

У Вани на фичах с 20 текстами на рандомфоресте и экстраТри 0.706 на кагле

Три модельки на маленьком количестве фичей 0.71702

Стекинг на 25 + 20 + 2 + 25 фичей от Севы и три модели  скор = 0.6992. Наверное из-за текстов

Стекинг на маленьком количестве фичей скор = 0.71886

Без стекинга Голосование из 4 моделей: RandomForest, ExtraTrees, Градиентный бустинг, XGBoost 25 + 25 + 2 фичей  скор на кагле = 0.72434

Голосование над стекингом и еще 4 модели дает 0.72298 на 10 tf-idf фичах

Голосование из 5 моделей: Stacking, RandomForest, ExtraTrees, Градиентный бустинг, XGBoost 25 + 25 + 2 фичей скор на кагле = 0.7270

##### Стекинг над 4 моделями 7 tf-idf фичей дал 0.734

Голосование над 5 моделями включая стекинг дало 0.83 на валидации и 0.731 на кэгле. Это 15 фичей от Севы и 10 tf_idf

Стекинг без голосования дало 0.7344 на кэгле, а на валидации 0.7688 на 5 tf-idf фичах и 25 Севиных 

##### Стекинг на катбусте и еще 3 модели, final_esimator = MLPClassifier() на 15 фичах от Севы и 10 tf-idf на валидации 0.831, а на кэгле 0.73644

In [242]:
!pip install autokeras

     |████████████████████████████████| 164 kB 975 kB/s eta 0:00:01
     |████████████████████████████████| 62 kB 721 kB/s eta 0:00:01
  Using cached future-0.18.2.tar.gz (829 kB)
  Using cached tabulate-0.8.7-py3-none-any.whl (24 kB)


  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-py3-none-any.whl size=78936 sha256=5b8eacfd3d09dd47ffe5decadce22a2d57b62f0a46c588730b79fab7b168ed05
  Stored in directory: /Users/egor/Library/Caches/pip/wheels/78/e2/80/7fe373cad54ad22b06d0d6204cbc29cead9e69bb2680327775
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491058 sha256=7c6f55f8556c75aaa3a92189474a1ffb6f2ca6f8c73e8020606f772f5a8b3dda
  Stored in directory: /Users/egor/Library/Caches/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
  Created wheel for terminaltables: filename=terminaltables-3.1.0-py3-none-any.whl size=15354 sha256=9d9d12e13a5cd421f5588c17da0e5c9c02f47e7ed7c9da558a50df822f5241d9
  Stored in directory: /Users/egor/Library/Caches/pip/wheels/ba/ad/c8/2d98360791161cd3db6daf6b5e730f34021fc9367d5879f497
Successfully built keras-tuner future terminaltables
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.7/bin/python3.7 -m 

In [243]:
import autokeras as ak

In [244]:
X_train.shape

(11690, 36)

In [245]:
y_train.shape

(11690,)

In [246]:
data = pd.DataFrame()

In [253]:
y = np.array(y_train)[:,np.newaxis]
y.shape

(11690, 1)

In [257]:
Train_df = pd.DataFrame(data=X_y)

In [258]:
Train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11690 entries, 0 to 11689
Data columns (total 37 columns):
0     11690 non-null float64
1     11690 non-null float64
2     11690 non-null float64
3     11690 non-null float64
4     11690 non-null float64
5     11690 non-null float64
6     11690 non-null float64
7     11690 non-null float64
8     11690 non-null float64
9     11690 non-null float64
10    11690 non-null float64
11    11690 non-null float64
12    11690 non-null float64
13    11690 non-null float64
14    11690 non-null float64
15    11690 non-null float64
16    11690 non-null float64
17    11690 non-null float64
18    11690 non-null float64
19    11690 non-null float64
20    11690 non-null float64
21    11690 non-null float64
22    11690 non-null float64
23    11690 non-null float64
24    11690 non-null float64
25    11690 non-null float64
26    11690 non-null float64
27    11690 non-null float64
28    11690 non-null float64
29    11690 non-null float64
30    11690 non-null 

In [259]:
Train_df.to_csv('train_df.csv')

In [263]:
Train_df.head()

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,4.0,4.0,3.0,3.0,3.0,2.0,2.0,2.0,2.0,2.0,...,0.614383,0.528873,0.509626,0.504905,0.501869,0.461926,0.449864,-1.0,0.0,0.0
1,8.0,7.0,7.0,7.0,7.0,7.0,4.0,4.0,3.0,3.0,...,0.653389,0.628512,0.580146,0.533492,0.457677,0.426910,0.407499,0.0,0.0,0.0
2,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,...,0.282926,0.263492,0.260648,0.248446,0.230717,0.230373,0.228871,-1.0,0.0,0.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.188717,0.179073,0.151745,0.146303,0.135194,0.132056,0.129261,-1.0,0.0,0.0
4,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.450056,0.424316,0.422547,0.414055,0.398026,0.359367,0.339476,-1.0,0.0,0.0


In [266]:
Train_df[36]

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
5        0.0
6        0.0
7        0.0
8        0.0
9        0.0
10       0.0
11       0.0
12       0.0
13       0.0
14       0.0
15       0.0
16       0.0
17       0.0
18       0.0
19       0.0
20       0.0
21       0.0
22       0.0
23       0.0
24       0.0
25       0.0
26       0.0
27       0.0
28       1.0
29       0.0
        ... 
11660    0.0
11661    0.0
11662    0.0
11663    1.0
11664    1.0
11665    0.0
11666    0.0
11667    0.0
11668    0.0
11669    0.0
11670    1.0
11671    1.0
11672    0.0
11673    0.0
11674    0.0
11675    0.0
11676    0.0
11677    0.0
11678    0.0
11679    0.0
11680    0.0
11681    0.0
11682    0.0
11683    0.0
11684    0.0
11685    0.0
11686    0.0
11687    0.0
11688    0.0
11689    0.0
Name: 36, Length: 11690, dtype: float64

In [338]:
clf = ak.StructuredDataClassifier(
    overwrite=True,
    max_trials=5)

In [339]:
clf.fit(X_train, y_train, epochs=20)

Trial 5 Complete [00h 00m 20s]
val_accuracy: 0.8517718315124512

Best val_accuracy So Far: 0.8517718315124512
Total elapsed time: 00h 01m 27s
INFO:tensorflow:Oracle triggered exit

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1/20
366/366 [==============================] - 1s 2ms/step - loss: 0.6339 - accuracy: 0.6887
Epoch 2/20
366/366 [==============================] - 1s 2ms/step - loss: 0.5514 - accuracy: 0.7407
Epoch 3/20
366/366 [==============================] - 1s 2ms/step - loss: 0.4903 - accuracy: 0.7815
Epoch 4/20
366/366 [========

In [340]:
X_test.shape

(16627, 36)

In [341]:
predicted_y = clf.predict(X_test)

In [342]:
predicted_y_train = clf.predict(X_train)

In [343]:
f1_score(predicted_y_train, y_train)

0.7126715092816788

In [344]:
predicted_y = predicted_y.reshape(-1, 1)

In [345]:
predicted_y = predicted_y.astype(int)

In [ ]:
predicted_y = np.array(predicted_y).ravel()

In [333]:
with open("/Users/egor/Desktop/resultBest.csv", "w", encoding='utf-8') as outfile:
    outfile.write(str("pair_id,target\n"))
    for i in range(len(y_test)):
        outfile.write(str(pair_ids[i]) + str(",") + str(predicted_y[i]) + str("\n"))